실제 categorical variable을 다운받을 생각을 하지 못 했다.

어쩔 수 없이 가짜로 데이터를 만들고 작업을 하자.

(Use Iris dataset otherwise.)

In [1]:
import numpy as np
import pandas as pd
import sys
sys.path.append("..")

from python.preprocess import *  #only for the categorical variables
from python.OREM import *
from sklearn.model_selection import train_test_split
from tqdm import tqdm

# Preprocess

In [2]:
type = "[Mixed]"
fn = "vowel0"

if type == "[Cate]":
    location = "categorical"
if type == "[Mixed]":
    location = "mixed"

data = pd.read_csv("../data/"+type+" "+fn+".csv", index_col=0)
data = check_class(data)
# You can see the printed statements have the same results with belows
#data.iloc[:, -1].value_counts()
display(data.head(3))
print(data.shape)

# of negative: 898
# of positive: 90
Majority Label changed: negative->0
Minority Label changed: positive->1


,1,2,3,4,5,6,7,8,9,10,11,12,LABEL
1,Andrew,Yes,-3.639,0.418,-0.670,1.779,-0.168,1.627,-0.388,0.529,-0.874,-0.814,1
2,Andrew,Yes,-3.327,0.496,-0.694,1.365,-0.265,1.933,-0.363,0.510,-0.621,-0.488,0
3,Andrew,Yes,-2.120,0.894,-1.576,0.147,-0.707,1.559,-0.579,0.676,-0.809,-0.049,0


(988, 13)


In [3]:
# Try this only for the mixed datasets
colLab = data.columns[-1]
tmp = pd.get_dummies(data, drop_first=True)

new = tmp[tmp.columns[tmp.columns.get_loc(colLab)+1:]].astype(int)
old = tmp[tmp.columns[:tmp.columns.get_loc(colLab)+1]]

data = pd.concat([new, old], axis=1)
display(data.head(3))
print(data.shape)

,1_Bill,1_David,1_Jo,1_Kate,1_Mark,1_Mike,1_Nick,1_Penny,1_Rich,1_Rose,...,4,5,6,7,8,9,10,11,12,LABEL
1,0,0,0,0,0,0,0,0,0,0,...,0.418,-0.670,1.779,-0.168,1.627,-0.388,0.529,-0.874,-0.814,1
2,0,0,0,0,0,0,0,0,0,0,...,0.496,-0.694,1.365,-0.265,1.933,-0.363,0.510,-0.621,-0.488,0
3,0,0,0,0,0,0,0,0,0,0,...,0.894,-1.576,0.147,-0.707,1.559,-0.579,0.676,-0.809,-0.049,0


(988, 26)


In [4]:
# Try this only for the categorical datasets
df = check_uniqueness(data.iloc[:, :-1])
df['Label'] = data.iloc[:, -1]
df.head(3)

There is at least one column which has different length of unique values.


,1_Bill,1_David,1_Jo,1_Kate,1_Mark,1_Mike,1_Nick,1_Penny,1_Rich,1_Rose,...,4,5,6,7,8,9,10,11,12,Label
1,0,0,0,0,0,0,0,0,0,0,...,0.418,-0.670,1.779,-0.168,1.627,-0.388,0.529,-0.874,-0.814,1
2,0,0,0,0,0,0,0,0,0,0,...,0.496,-0.694,1.365,-0.265,1.933,-0.363,0.510,-0.621,-0.488,0
3,0,0,0,0,0,0,0,0,0,0,...,0.894,-1.576,0.147,-0.707,1.559,-0.579,0.676,-0.809,-0.049,0


# OREM

In [5]:
minclass = 1
df_maj = df[df.iloc[:, -1].values != 1]
df_min = df[df.iloc[:, -1].values == 1]
ratio = 0.5

failed = []
for trial in tqdm(range(1, 101)):
    maj_train, maj_test = train_test_split(df_maj, test_size=ratio)
    min_train, min_test = train_test_split(df_min, test_size=ratio)

    train = pd.concat([maj_train, min_train])
    test = pd.concat([maj_test, min_test])

    label = train.iloc[:, -1]
    vc = label.value_counts()
    n_maj = vc[0]
    n_min = vc[1]

    train = train.iloc[:, :-1].copy()
    try:
        newDf, newLabel = OREM(train, list(label), n_maj - n_min)

        #아래 코드는 추가 확인이 필요합니다.
        newDf['Class'] = newLabel
        #newDf.Class.value_counts()     #check if you want to see if a balanced dataset is generated
        newDf.to_csv("../data/"+location+"/"+fn+"_train"+str(trial)+".csv", index=False)
        test.to_csv("../data/"+location+"/"+fn+"_test"+str(trial)+".csv", index=False)
    except:
        failed.append(trial)

  0%|          | 0/100 [00:00<?, ?it/s]

100%|██████████| 100/100 [00:28<00:00,  3.49it/s]
